In [1]:
import torch
import torch.nn as nn
import torch.optim as opt

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [3]:
mnist_dataset = datasets.MNIST(
    root="./data/",
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5, ), (0.5, ))
        ]
    )
)

In [4]:
batch_size = 64
epochs= 25

data_loader = torch.utils.data.DataLoader(
    dataset=mnist_dataset, batch_size=batch_size, shuffle=True, num_workers=2
)

In [99]:
# Output size = [(Input size - 1) * stride] - 2 * padding + kernel size

class Generator(nn.Module):
    def __init__(self, in_channels=100, out_channels=64, result_channels=1):
        super().__init__()
        self.layers = nn.Sequential(
            # 512 * 4 * 4
            nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels * 8, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(out_channels * 8),
            nn.ReLU(inplace=True),
            
            # 256 * 8 * 8
            nn.ConvTranspose2d(in_channels=out_channels * 8, out_channels=out_channels * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(out_channels * 4),
            nn.ReLU(inplace=True),
            
            # 128 * 16 * 16
            nn.ConvTranspose2d(in_channels=out_channels * 4, out_channels=out_channels * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(out_channels * 2),
            nn.ReLU(inplace=True),
            
            # 64 * 32 * 32
            nn.ConvTranspose2d(in_channels=out_channels * 2, out_channels=out_channels, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(out_channels * 2),
            nn.ReLU(inplace=True),
            
            # 1 * 28 * 28
            nn.ConvTranspose2d(in_channels=out_channels, out_channels=result_channels, kernel_size=1, stride=1, padding=2),
            nn.Tanh()    
        )

    def forward(self, x):
        return self.layers(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=1, out_channels=64):
        super().__init__()
        self.layers = nn.Sequential(
            # 64 * 14 * 14
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 128 * 7 * 7
            nn.Conv2d(out_channels, out_channels * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 256 * 3 * 3
            nn.Conv2d(out_channels * 2, out_channels * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 512 * 1 * 1
            nn.Conv2d(out_channels * 4, 1, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Sigmoid()              
        )

    def forward(self, x):
        return self.layers(x)

In [100]:
G = Generator()
G.cuda()
optimizer_g = opt.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

D = Discriminator()
D.cuda()
optimizer_d = opt.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

loss_function = nn.BCELoss()

real_label = 1
fake_label = 0

epochs = 25

for epoch in range(1, epochs + 1):
    for i, (x, _) in enumerate(data_loader):
        
        D.zero_grad()
        x = x.cuda()
        
        label = torch.full((batch_size, 1, 1, 1), real_label, dtype=torch.float).cuda()
        
        output = D(x)
        
        real_loss = loss_function(output, label).cuda()
        real_loss.backward()
        # print(output.mean().item())
        
        break
    break
        
        